In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import sys
import glob
from dotenv import load_dotenv, find_dotenv

# Load the .env file
load_dotenv(find_dotenv())

package_path = os.getenv('PACKAGE_PATH')
# package_path = '/home/dwna/projects/domain_class'
sys.path.append(package_path)


import pandas as pd
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')

from src.features.build_features import BuildFeatures


In [3]:
pd.set_option('display.max.colwidth', None,
              'display.max_columns', None,
)



In [4]:
def get_sheet_names(filepath):
    workbook = load_workbook(filepath, read_only=True)
    return workbook.sheetnames

In [5]:
def read_and_merge_sheets(file_path):
    # List of specific sheet names you want to check (strip spaces for each)
    normal_sheet, err_sheet = '(룰설정)도메인', '(진단실행)진단항목오류정보'
    specific_sheet_names = [normal_sheet, err_sheet]
    
    # Get the sheet names from the file
    sheet_names = get_sheet_names(file_path)
    sheet_names_set = set(name.strip() for name in sheet_names)
    
    # Find the intersection of specific sheet names and actual sheet names in the file
    existing_sheets = set(specific_sheet_names) & sheet_names_set
    
    # Print results
    print("file_path:", file_path)
    print("Existing Specific Sheets:", existing_sheets)
    
    # DataFrames dictionary
    dataframes = {}

    # Check if two or more specific sheet names exist and read them into DataFrames
    if len(existing_sheets) == 2:
        print("Two or more specific sheet names exist in the file. Creating DataFrames for each.")
        for sheet_name in existing_sheets:
            dataframes[sheet_name] = pd.read_excel(file_path, sheet_name=sheet_name)
    else:
        print("Less than two specific sheet names exist in the file. No DataFrames created.")
        return None
        
    df_nor, df_err = dataframes[normal_sheet], dataframes[err_sheet]
    
    nor_columns = ['table', 'col_nm', 'datatype', 'rule', 'domain', 'verify_type', 'err_free_data', 'act_date',
        'normal_data', 'others']
    err_columns = ['table', 'col_nm', 'domain', 'rule', 'total_cnt', 'err_cnt', 'err_rt', 'err_ext_sql',
       'err_data']
    
    df_err.columns = err_columns
    df_nor.columns = nor_columns
    
    columns = ['table', 'col_nm', 'domain', 'normal_data', 'err_data', 'datatype', 'rule',  'total_cnt', 'err_cnt', 'err_rt']
    df = df_nor.merge(df_err, 'left', on=['table','col_nm'],suffixes=('','_y')) 
    
    df = (df[columns]
        [~df.normal_data.isna()]
        .assign(normal_data = df.normal_data.astype(str).str.split(',').apply(lambda x:[] if x ==['nan'] else x), 
                err_data = df.err_data.astype(str).str.split(',').apply(lambda x:[] if x ==['nan'] else x)
               )
    )
     
    return df
   
    


In [6]:
def process_all_files(directory_path):
    # Find all .xlsx files in the directory
    all_files = glob.glob(directory_path + '/*.xlsx')
    
    # List to hold dataframes
    all_dataframes = []

    # Process each file
    for file in all_files:
        merged_df = read_and_merge_sheets(file)
        if merged_df is not None:
            all_dataframes.append(merged_df)

    # Concatenate all dataframes
    final_dataframe = pd.concat(all_dataframes, axis=0, ignore_index=True)
    return final_dataframe

In [7]:
df = process_all_files(package_path + '/data/raw/xlsx')

file_path: /home/dwna/projects/domain_class/data/raw/xlsx/(최종)[SDQ_REPORT]RESULT_VALUE_비점오염원관리정보시스템DB.xlsx
Existing Specific Sheets: {'(룰설정)도메인', '(진단실행)진단항목오류정보'}
Two or more specific sheet names exist in the file. Creating DataFrames for each.
file_path: /home/dwna/projects/domain_class/data/raw/xlsx/(SDQ_REPORT)RESULT_VALUE_올바로_2차_최종.xlsx
Existing Specific Sheets: {'(룰설정)도메인', '(진단실행)진단항목오류정보'}
Two or more specific sheet names exist in the file. Creating DataFrames for each.
file_path: /home/dwna/projects/domain_class/data/raw/xlsx/0901.[SDQ_REPORT]RESULT_VALUE_폐기물처리현장정보_2차_최종.xlsx
Existing Specific Sheets: {'(룰설정)도메인', '(진단실행)진단항목오류정보'}
Two or more specific sheet names exist in the file. Creating DataFrames for each.
file_path: /home/dwna/projects/domain_class/data/raw/xlsx/(최종)[SDQ_REPORT]RESULT_VALUE_가축분뇨전자인계관리DB.xlsx
Existing Specific Sheets: {'(룰설정)도메인', '(진단실행)진단항목오류정보'}
Two or more specific sheet names exist in the file. Creating DataFrames for each.
file_path: /home/dwna/pro

In [17]:
package_path

'/home/dwna/projects/domain_class'

In [19]:
df.to_csv(package_path+'/data/external/source.csv',index=False)

In [21]:
df.to_excel?

Signature:
df.to_excel(
    excel_writer,
    sheet_name: 'str' = 'Sheet1',
    na_rep: 'str' = '',
    float_format: 'str | None' = None,
    columns: 'Sequence[Hashable] | None' = None,
    header: 'Sequence[Hashable] | bool_t' = True,
    index: 'bool_t' = True,
    index_label: 'IndexLabel' = None,
    startrow: 'int' = 0,
    startcol: 'int' = 0,
    engine: 'str | None' = None,
    merge_cells: 'bool_t' = True,
    inf_rep: 'str' = 'inf',
    freeze_panes: 'tuple[int, int] | None' = None,
    storage_options: 'StorageOptions' = None,
) -> 'None'
Docstring:
Write object to an Excel sheet.

To write a single object to an Excel .xlsx file it is only necessary to
specify a target file name. To write to multiple sheets it is necessary to
create an `ExcelWriter` object with a target file name, and specify a sheet
in the file to write to.

Multiple sheets may be written to by specifying unique `sheet_name`.
With all data written to the file it is necessary to save the changes.
Note that

In [23]:
df.to_excel(package_path+'/data/external/source.xlsx', index=False)

In [9]:
df.normal_data.apply(len)

0        5
1        5
2        5
3        5
4        5
        ..
10100    3
10101    5
10102    1
10103    1
10104    1
Name: normal_data, Length: 10105, dtype: int64

In [10]:
import random
import re
import string

def is_korean_char(ch):
    """Check if a character is a Korean character."""
    return '\uAC00' <= ch <= '\uD7A3'

def get_random_char(char_type):
    """Get a random character based on the type (digit, English alphabet, Korean character)."""
    if char_type == 'digit':
        return str(random.randint(0, 9))
    elif char_type == 'english':
        return random.choice(string.ascii_letters)
    elif char_type == 'korean':
        return chr(random.randint(0xAC00, 0xD7A3))
    
def find_pattern(s):
    keys = ["yn", "date_time", "number", "integer", "bunho", "email", "url"]
    patterns = {key: BuildFeatures.patterns[key] for key in keys}
    
    for key, pattern in patterns.items():
        if re.match(pattern, s, re.IGNORECASE):
            return pattern
    else:
        return None
        

def modify_string(s):
    """Modify a string by randomly replacing a character while ensuring it matches a regex pattern."""
    def get_char_type(ch):
        """Determine the character type."""
        if ch.isdigit():
            return 'digit'
        elif ch.isalpha() and ch.isascii():
            return 'english'
        elif is_korean_char(ch):
            return 'korean'
        else:
            return 'other'

    length = len(s)
    pattern = find_pattern(s)
    attempts = 0

    while attempts < 1000:  # Limit attempts to avoid infinite loops
        # Randomly pick an index
        index = random.randint(0, length - 1)

        char_type = get_char_type(s[index])
        # Skip if the character type is 'other'
        if char_type == 'other':
            attempts += 1
            continue

        # Replace the character at the selected index
        new_char = get_random_char(char_type)
        new_string = s[:index] + new_char + s[index + 1:]
        
        if pattern is None:
            return new_string

        # Check if the new string matches the pattern
        if re.match(pattern, new_string, re.IGNORECASE):
            return new_string

        attempts += 1

    return None  # Return None if no match is found within the attempt limit

# Example usage
# input_string = "Hello 123 안녕하세요"
# pattern = r"^[A-Za-z0-9\s]+안녕하세요$"  # Example pattern

# modified_string = modify_string(input_string, pattern)
# if modified_string:
#     print("Modified String:", modified_string)
# else:
#     print("No matching string found within the attempt limit.")


def generate_combined_set(normal_list, abnormal_list, abnormal_probability, length):
    # Early return if both lists are empty
    if not normal_list and not abnormal_list:
        return []

    # Adjust counts if one of the lists is empty
    abnormal_count = int(length * abnormal_probability) if abnormal_list else 0
    normal_count = length - abnormal_count

    # Generate lists based on the counts
    chosen_abnormal = random.choices(abnormal_list, k=abnormal_count) if abnormal_list else []
    chosen_normal = random.choices(normal_list, k=normal_count) if normal_list else []
    
    if chosen_normal:
        chosen_normal = [modify_string(s) for s in chosen_normal if s]

    # Combine and shuffle the final list
    final_list = chosen_normal + chosen_abnormal
    random.shuffle(final_list)

    return final_list


def profile_data(df: pd.DataFrame, abnormal_rate: float, length: int)->pd.DataFrame:
    """
    Profile the data from a DataFrame using the BuildFeatures class.
    """
    profiles = []

    for row in df.itertuples(index=False):
        combined = generate_combined_set(row.normal_data, row.err_data, abnormal_rate, length)
        series = pd.Series(combined, name=row.col_nm)

        profile = BuildFeatures(series, domain=row.domain).profiling_patterns()
        profiles.append(profile)
    
    return pd.DataFrame(profiles).dropna()


# profiles = profile_data(df, abnormal_rate=0.01, length=1000)


from joblib import Parallel, delayed
import dask.dataframe as dd

def profile_row(row, abnormal_rate, length):
    warnings.filterwarnings('ignore')  # multiprocessing시 warning 출력 안되게 하기위해 설정함.
    combined = generate_combined_set(row.normal_data, row.err_data, abnormal_rate, length)
    series = pd.Series(combined, name=row.col_nm)
    profile = BuildFeatures(series, domain=row.domain).profiling_patterns()
    return profile

def profile_data_joblib(df: pd.DataFrame, abnormal_rate: float, length: int, num_cores:int=-1) -> pd.DataFrame:
    # Prepare the data for parallel processing
    results = Parallel(n_jobs=num_cores)(delayed(profile_row)(row,abnormal_rate, length) 
                                  for row in df.itertuples(index=False))
    return pd.DataFrame(results).dropna()

def profile_data_dask(df: pd.DataFrame, abnormal_rate: float, length: int, number_of_partitions:int =10) -> pd.DataFrame:
    # Prepare the data for parallel processing
    ddf = dd.from_pandas(df, npartitions=number_of_partitions)
    result = ddf.map_partitions(lambda partition: partition.apply(lambda row: profile_row(row,  abnormal_rate, length), axis=1)) \
                .compute(scheduler='processes').dropna()

    return result


In [11]:


profiles = profile_data(df[:10], abnormal_rate=0.01, length=10)


In [12]:
profiles

,col_name,yn,date_time,number,integer,bunho,email,url,part_num,part_text,part_discriminator,part_mask,part_minus,len_purity,value_nunique,value_distr,datatype,BUNHO,NALJJA,MYEONG,JUSO,YEOBU,CODE,ID,SURYANG,GEUMAEK,NAEYOUNG,YUL,ETC,domain
0,LD_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9,3.121928,object,1,0,0,0,0,0,0,0,0,0,0,0,번호
1,AD_LEV,0.2,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7,2.721928,object,0,0,0,0,0,0,0,0,0,0,0,1,코드
2,CTY_SGG_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7,2.646439,object,1,0,0,0,0,0,0,0,0,0,0,0,번호
3,CTY_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,10,3.321928,object,1,0,0,0,0,0,0,0,0,0,0,0,번호
4,SGG_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9,3.121928,object,1,0,0,0,0,0,0,0,0,0,0,0,번호
5,EMD_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9,3.121928,object,1,0,0,0,0,0,0,0,0,0,0,0,번호
6,DL_NO,0.0,0.0,0.9,0.9,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9,3.121928,object,1,0,0,0,0,0,0,0,0,0,0,0,번호


In [ ]:


profiles = profile_data(df, abnormal_rate=0.01, length=1000)


In [59]:

df_profiled = profile_data_dask(df, abnormal_rate=0.01, length=1000, number_of_partitions=5)


In [13]:

profiles_job = profile_data_joblib(df, abnormal_rate=0.01, length=1000)

In [14]:
profiles_job

,col_name,yn,date_time,number,integer,bunho,email,url,part_num,part_text,part_discriminator,part_mask,part_minus,len_purity,value_nunique,value_distr,datatype,BUNHO,NALJJA,MYEONG,JUSO,YEOBU,CODE,ID,SURYANG,GEUMAEK,NAEYOUNG,YUL,ETC,domain
0,LD_NO,0.000,0.0,1.000,1.000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,367,8.072534,object,1,0,0,0,0,0,0,0,0,0,0,0,번호
1,AD_LEV,0.350,0.0,1.000,1.000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,10,3.227319,object,0,0,0,0,0,0,0,0,0,0,0,1,코드
2,CTY_SGG_NO,0.000,0.0,1.000,1.000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,207,7.384592,object,1,0,0,0,0,0,0,0,0,0,0,0,번호
3,CTY_NO,0.000,0.0,1.000,1.000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,50,5.112590,object,1,0,0,0,0,0,0,0,0,0,0,0,번호
4,SGG_NO,0.000,0.0,1.000,1.000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,92,6.180919,object,1,0,0,0,0,0,0,0,0,0,0,0,번호
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10100,HOUR,0.000,0.0,0.464,0.464,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,37,4.957844,object,0,0,0,0,0,0,0,0,0,0,0,1,날짜
10101,MIN,0.000,0.0,0.888,0.888,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,55,5.237934,object,0,0,0,0,0,0,0,0,0,0,0,1,날짜
10102,USE_YN,0.511,0.0,0.000,0.000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4,1.998756,object,0,0,0,0,1,0,0,0,0,0,0,0,여부
10103,RGSTER_ID,1.000,0.0,1.000,1.000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2,0.999584,object,0,0,0,0,0,0,1,0,0,0,0,0,번호


In [15]:
profiles_job.to_csv(package_path + '/data/processed/profiles/1/ver_1_len_1000_rate_0.01.cvs', index=False)

In [30]:
profile_data(df.query('col_nm =="LD_NO"'), abnormal_rate=0.01, length=10)

,col_name,yn,date_time,number,integer,bunho,email,url,part_num,part_text,part_discriminator,part_mask,part_minus,len_purity,value_nunique,value_distr,datatype,BUNHO,NALJJA,MYEONG,JUSO,YEOBU,CODE,ID,SURYANG,GEUMAEK,NAEYOUNG,YUL,ETC,domain
0,LD_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,10,3.321928,object,True,0,0,0,0,0,0,0,0,0,0,0,번호


In [15]:

profiles_job = profile_data_joblib(df, abnormal_rate=0.01, length=10)

In [16]:
profiles_job

,col_name,yn,date_time,number,integer,bunho,email,url,part_num,part_text,part_discriminator,part_mask,part_minus,len_purity,value_nunique,value_distr,datatype,BUNHO,NALJJA,MYEONG,JUSO,YEOBU,CODE,ID,SURYANG,GEUMAEK,NAEYOUNG,YUL,ETC,domain
0,LD_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,9,3.121928,object,True,0,0,0,0,0,0,0,0,0,0,0,번호
1,AD_LEV,0.4,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,5,2.170951,object,0,0,0,0,0,0,0,0,0,0,0,True,코드
2,CTY_SGG_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,9,3.121928,object,True,0,0,0,0,0,0,0,0,0,0,0,번호
3,CTY_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,9,3.121928,object,True,0,0,0,0,0,0,0,0,0,0,0,번호
4,SGG_NO,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,10,3.321928,object,True,0,0,0,0,0,0,0,0,0,0,0,번호
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10100,HOUR,0.0,0.0,0.7,0.7,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,8,2.846439,object,0,0,0,0,0,0,0,0,0,0,0,True,날짜
10101,MIN,0.0,0.0,0.9,0.9,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,8,2.921928,object,0,0,0,0,0,0,0,0,0,0,0,True,날짜
10102,USE_YN,0.6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4,1.970951,object,0,0,0,0,True,0,0,0,0,0,0,0,여부
10103,RGSTER_ID,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2,0.881291,object,0,0,0,0,0,0,True,0,0,0,0,0,번호


### data review

In [58]:
pd.get_dummies(profiles_job.BUNHO, drop_first=True)

,True
0,True
1,False
2,True
3,True
4,True
...,...
10100,False
10101,False
10102,False
10103,False


In [21]:
df

,table,col_nm,domain,normal_data,err_data,datatype,rule,total_cnt,err_cnt,err_rt
0,NPCOOWN.CMAD01,LD_NO,번호,"[1100000000, 1111000000, 1111010100, 1111010200, 1111010300]",[],VARCHAR2(10),형식_숫자10자리(9999999999),"20,542",0,0.0
1,NPCOOWN.CMAD01,AD_LEV,코드,"[4, 3, 5, 2, 1]",[],VARCHAR2(1),주소_레벨,"20,542",0,0.0
2,NPCOOWN.CMAD01,CTY_SGG_NO,번호,"[47250, 47170, 47230, 48890, 41590]",[],VARCHAR2(10),형식_숫자5자리(99999),"20,542",0,0.0
3,NPCOOWN.CMAD01,CTY_NO,번호,"[47, 46, 48, 44, 41]",[],VARCHAR2(10),형식_숫자2자리(99),"20,542",0,0.0
4,NPCOOWN.CMAD01,SGG_NO,번호,"[150, 710, 110, 130, 770]",[],VARCHAR2(10),형식_숫자3자리(999),"20,542",0,0.0
...,...,...,...,...,...,...,...,...,...,...
10100,WDSUSER.T_ST_STATS_MANAGE_SETUP,HOUR,날짜,"[02, 01, 03]",[],VARCHAR2(2),시,10,0,0.0
10101,WDSUSER.T_ST_STATS_MANAGE_SETUP,MIN,날짜,"[00, 20, 40, 10, 30]",[],VARCHAR2(2),분,10,0,0.0
10102,WDSUSER.T_ST_STATS_MANAGE_SETUP,USE_YN,여부,[Y],[],VARCHAR2(1),여부Y/N,10,0,0.0
10103,WDSUSER.T_ST_STATS_MANAGE_SETUP,RGSTER_ID,번호,[1],[],NUMBER(10),숫자만사용,10,0,0.0


In [22]:
df[df.col_nm=='USE_YN'].normal_data.value_counts()

normal_data
[Y]          39
[Y, N]       27
[N, Y]        5
[Y, N, D]     3
[1, 0]        1
Name: count, dtype: int64

In [ ]:
df[df.rule.astype(str).str.contains("이메일")]

,table,col_nm,domain,normal_data,err_data,datatype,rule,total_cnt,err_cnt,err_rt
50,NPCOOWN.CMCA01,EML,번호,"[NULL, -, test@test.com, bsoh@korbi.com, healim5194@keco.or.kr]",[],VARCHAR2(30),이메일,135,0,0.00
575,NPFCOWN.FCDA11,CRP_EMI,번호,"[NULL, bsh1125@korea.kr, big4smile@korea.kr, future316@korea.kr, arim@korea.kr]",[],VARCHAR2(30),이메일,48,0,0.00
749,NPFCOWN.FCMA23,EMI,번호,"[NULL, bsh1125@korea.kr, big4smile@korea.kr, future316@korea.kr, arim@korea.kr]",[],VARCHAR2(100),이메일,48,0,0.00
3826,AIUSER.MIG_CM_CMPY_GUNSEOL_TRTM_605,E_MAIL,번호,"[NULL, nambu21@naver.com, sje06070@naver.com, chowoneng@naver.com, connect_4@daum.net]","[000000@000.000, chosw427]",VARCHAR2(100),이메일,569,2,0.35
4466,AIUSER.T_CM_USER_CHG_REQST,EMAIL,번호,"[dc0633@naver.com, daelim4489@naver.com, dosungena@daum.net, greencmh@hanmail.net, kbi4557@nate.com]",[],VARCHAR2(100),이메일,386,0,0.00
4480,AIUSER.T_CM_USER_HIST,EMAIL,번호,"[greencmh@hanmail.net, tomato0722@naver.com, 4178114701@hanmail.net, aaa@hanmail.net, b012b@hanmail, net]",[],VARCHAR2(100),이메일,50,0,0.00
8101,LSNSUSER.TB_USCM,EMAIL,번호,"[a@naver.com, 1@naver.com, 000@naver.com, @, keco@naver.com]","[ㄴ@naver.com, 00000@00000, 1@1, aa@aa, d@naver.com, mino85@kra.co.kr, .@., 0000000@0000, 000000@00000, 00000@0000]",VARCHAR2(100),이메일,"11,001",41,0.37
8133,LSNSUSER.TB_USER,EMAIL,번호,"[a@naver.com, 000@naver.com, 1@naver.com, qnssy@naver.com, skjfor@nate.com]","[@, mino85@kra.co.kr, jung_su_@naver.com, rock-kasu@, ㅣ8684y@hanmail.net]",VARCHAR2(100),이메일,"4,235",7,0.17
8368,WDSUSER.T_CM_BBS_REPLY,EMAIL,번호,"[samchangeng5123@naver.com, jumong77777@nate.com, jyw5474@naver.com, oatmnc@naver.com, sgr6706@naver.com]",[],VARCHAR2(100),이메일,139,0,0.00
8509,WDSUSER.T_CM_CRTFCT_ISSU_REQST,JOB_CHARGER_EMAIL,번호,"[bondangri@daum.net, ja8217@keco.or.kr, nadohun2@keco.or.kr]",[],VARCHAR2(100),이메일,4,0,0.00


In [73]:
profile_dataset = profiles_job[profiles_job.domain.astype(str) != 'nan']

In [77]:
profile_dataset.value_nunique.value_counts()

value_nunique
5     3520
4     1840
2     1135
1     1131
3      511
6      105
11      86
10      71
12      70
9       55
8       43
7       42
13      20
14       4
Name: count, dtype: int64

# FYI

In [69]:
import openpyxl
import pandas as pd

file_path = "quality_evaluation_data.xlsx"
df_e = pd.read_excel(file_path, sheet_name='error_samples')
df_n = pd.read_excel(file_path, sheet_name='normal_samples')
err_columns = ['table', 'col_nm', 'domain', 'rule', 'total_cnt', 'err_cnt', 'err_rt', 'err_ext_sql',
       'err_data']
nor_columns = ['table', 'col_nm', 'datatype', 'rule', 'domain', 'verify_type', 'err_free_data', 'act_date',
       'nornal_data', 'others']
df_e.columns = err_columns
df_n.columns = nor_columns
df = df_n.merge(df_e, 'left', on=['table','col_nm'])
df = df[['table', 'col_nm', 'domain_x', 'nornal_data', 'err_data', 'datatype', 'rule_x',  'total_cnt', 'err_cnt', 'err_rt']]
df =df.rename(columns={'nornal_data':'normal_data', 'rule_x':'rule', 'domain_x':'domain'})
df = df[~df.normal_data.isna()]
df = df.assign(normal_data = df.normal_data.str.split(',').astype('str'),
             err_data = df.err_data.str.split(',').astype('str'),
)

In [56]:
# build_feature.py
import pandas as pd
import numpy as np
import re
from pandas import Series


class BuildFeatures():
    """
    ## 전처리
    - None, NULL 등의 갯수/비율 체크 및 삭제
    ##전체패턴
    - 날짜 형식 비율 
    - 숫자 비율(실수, 정수, 00012같은 것은 걸려낼 것)
    - 정수 비율
    - 번호패턴 비율
    - 문자번호패턴 비율 
    - 이메일 비율
    - URL 비율
    ##부분패턴
    - 숫자포함
    - 문자포함(한글영문)
    - discriminator포함
    - masking포함
    - 음수포함
    ## 데이터 속성
    - 데이터길이 Purity : 0~1, 1에 가까울수록 고정길이, (엔트로피를 사용해도 될 듯)
    - 데이터 Distinct 수
    - 데이터 엔트로피 : distinct 값의 분포. Noise로 인한 distinct가 크면 의미가 있는데, 계산량이 많이 들 수 있다.
    ##메타데이터
    - 컬럼도메인: 컬럼명으로부터 도메인 접미사 추출하여 해당도메인 onehot처리(도메인별 접미사 구성)
    - 데이터 타입
    
    # usage
        data = {
            'sample_column': [
                '2023-01-01', '100', '300.5', 'test@example.com', 'http://example.com', 
                '2023-02-01', '200', '400.5', 'hello@world.com', 'https://world.com',
                '2023-03-01', 'NULL', None, 'NaN', '2023-04-01'
            ]
        }
        # 판다스 데이터프레임 생성
        df = pd.DataFrame(data)
        # BuildFeatures 클래스 인스턴스화
        bf = BuildFeatures(df['sample_column'], 'SAMPLE_sz')
        # 프로파일링 패턴 호출
        profile = bf.profiling_patterns()
        print(profile)    
            
    """
    YYYY = r"(19|20)\d{2}"
    MM = r"(0[1-9]|1[012])"
    DD = r"(0[1-9]|[12][0-9]|3[01])"
    TM = r"\s+([01][0-9]|2[0-4]):[0-5][0-9](:[0-5][0-9])?(\s+(PM|AM))?"
    patterns = {
        "date_time": fr"^({YYYY}[-./]{MM}[-./]{DD}|{MM}[-./]{DD}[-./]{YYYY}|{DD}[-./]{MM}[-./]{YYYY})({TM})?$",
        "number": r"^(?!0\d)\d+([.]\d*)?$",
        "integer": r"^(?!0\d)\d+$",
        "bunho": r"^[A-Za-z0-9\uAC00-\uD7A3]+([-./:][A-Za-z0-9\uAC00-\uD7A3]+)*$",
        "email": r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$",
        "url": r"^(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?$",
        "part_num": r"\d+",
        "part_text": r"[A-Za-z\uAC00-\uD7A3]+",
        "part_discriminator": r"[./-:]",
        "part_mask": r"[#*]{3,}",
        "part_minus": r"^-\d",
    }

    #  도메인 접미사 패턴 그룹 정의 및 결합
    suffix_patterns = {
        "BUNHO": "(?P<BUNHO>(NO|SN|ZIP|TKN|VIN|ENTN)$)",
        "NALJJA": "(?P<NALJJA>(DT|YMD|YM|YR|SYR|MM|DAY)$)",
        "MYEONG": "(?P<MYEONG>(NM|TTL)$)",
        "JUSO": "(?P<JUSO>(ADDR)$)",
        "YEOBU": "(?P<YEOBU>YN$)",
        "CODE": "(?P<CODE>CD$)",
        "ID": "(?P<ID>ID$)",
        "SURYANG": "(?P<SURYANG>(LOT|DONT|GRD|LVL|GFA|AREA|PRG|SCR|CNT|LEN|SZ)$)",
        "GEUMAEK": "(?P<GEUMAEK>(AMT|FEE|UNTPRC)$)",
        "NAEYOUNG": "(?P<NAEYOUNG>CN$)",
        "YUL": "(?P<YUL>RT$)",
    }
    combined_pattern = re.compile("|".join(suffix_patterns.values()),  re.IGNORECASE)
    
    def __init__(self, series:Series, col_name:str=None):
        if not isinstance(series, pd.Series):
            raise ValueError("series must be a pandas Series")        
        
        na_rate = series.isna().mean()
        null_rate = series.str.strip().isin(['NULL', 'NaN']).mean()
        
        self.null_rate = na_rate + null_rate
        self.series = series[~series.isin(['NULL', 'NaN'])].dropna()
        self.datatype = self.series.dtype.name  
        self.col_name = col_name if col_name else self.series.name

        # self.patterns = patterns
        
    def rate_matching_pattern(self, pattern:str)-> float:
        # series = self.series.dropna()
        return self.series.str.contains(pattern, regex=True).mean()
    

    def get_length_purity(self)-> float:
        # series = self.series.dropna().astype(str)
        ratio = self.series.str.len().value_counts(normalize=True)
        length_purity = (ratio * ratio).sum()
        return length_purity
    
    def get_value_nunique(self):
        return self.series.nunique()
        # self.series.value_counts()
        
    def get_value_distr(self):
        ratio = self.series.value_counts(normalize=True)
        entropy = (-ratio * np.log2(ratio)).sum()
        return entropy
    
    def find_suffix_domain(self):
        """
        컬럼명에서 도메인분류어 패턴을 추출하여 어떤 도메인그룹의 suffix에 해당하는지 검출
        :return: 매칭된 그룹 이름 또는 '해당 사항 없음'
        """
        word = self.col_name
        if not isinstance(word, str) or not word:
            return "유효하지 않은 입력"
        
        match = self.combined_pattern.search(word)
        if match:
            matched_groups = [name for name, value in match.groupdict().items() if value]
            return matched_groups if matched_groups else "ETC"
        else:
            return "ETC"
           
    def one_hot_encode(self, domain:str, categories: list) -> dict:
       """
       주어진 pandas Series를 one-hot 인코딩으로 변환합니다.
    
       :param series: 인코딩할 pandas Series 객체
       :param categories: one-hot 인코딩을 위한 카테고리 리스트
       :return: 카테고리 값을 키로 하고 one-hot 인코딩된 pd.Series를 값으로 하는 딕셔너리
       """
       if not isinstance(categories, list):
           categories = list(categories)
       categories = categories +['ETC'] # categories에 들어가지 않는 것은 ETC로 추가로 분류한다.
       one_hot_encoded = pd.get_dummies(domain, prefix='', prefix_sep='').reindex(columns=categories, fill_value=0)
       return one_hot_encoded.iloc[0].to_dict() 

    def profiling_patterns(self)-> Series:
        features = {}
        for key, pattern in self.patterns.items():
            features[key] = self.rate_matching_pattern(pattern)
        features['len_purity'] = self.get_length_purity()
        features['value_nunique'] = self.get_value_nunique()
        features['value_distr'] = self.get_value_distr()
        features['datatype'] = self.datatype
        suffix_domain = self.find_suffix_domain()
        one_hot_encoded = self.one_hot_encode(suffix_domain, self.suffix_patterns.keys()) 
        features = {**features, **one_hot_encoded}
        
        return pd.Series(features,name = self.col_name )
    


In [ ]:
### use_case
data = {
    'sample_column': [
        '2023-01-01', '100', '300.5', 'test@example.com', 'http://example.com', 
        '2023-02-01', '200', '400.5', 'hello@world.com', 'https://world.com',
        '2023-03-01', 'NULL', None, 'NaN', '2023-04-01'
    ]
}


data = {

    'ENTN': [
        '200905830','201785062','080146287','200100957','201810936'
    ]
}

# 판다스 데이터프레임 생성
df = pd.DataFrame(data)
# BuildFeatures 클래스 인스턴스화
bf = BuildFeatures(df['ENTN'], 'ENTN')
# 프로파일링 패턴 호출
profile = bf.profiling_patterns()
print(profile)    


In [83]:
df.normal_data[0]

"['080068544', '080134675', '080135855', '080139164', '080147596']"

In [84]:
def str2series(str_list):
    return pd.Series(eval(str_list))

In [87]:
df.columns

Index(['table', 'col_nm', 'domain', 'normal_data', 'err_data', 'datatype',
       'rule', 'total_cnt', 'err_cnt', 'err_rt'],
      dtype='object')

In [88]:
profiles =[]
for row in df[:10].itertuples():
    series = str2series(row.normal_data)
    series.name = row.col_nm
    profile = BuildFeatures(series).profiling_patterns()
    profiles.append(profile)
    

/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return self.series.str.contains(pattern, regex=True).mean()
/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return self.series.str.contains(pattern, regex=True).mean()
/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return self.series.str.contains(pattern, regex=True).mean()
/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return self.series.str.contains(pattern, regex=True).mean()
/tmp/ipykernel_4616/930959982.py:101: UserWarning: This pattern is i

In [91]:
type(profile)

pandas.core.series.Series

In [96]:
pd.DataFrame(profiles)

,date_time,number,integer,bunho,email,url,part_num,part_text,part_discriminator,part_mask,...,MYEONG,JUSO,YEOBU,CODE,ID,SURYANG,GEUMAEK,NAEYOUNG,YUL,ETC
ENTN,0.0,0.00,0.00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
GRADE,0.0,1.00,1.00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
PWD,0.0,0.80,0.80,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
USNA,0.0,0.00,0.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,True
APPR_GIVE_DATE,1.0,0.00,0.00,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
OPER_GIVE_DATE,1.0,0.00,0.00,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
OPER_GIVE_NAME,0.0,0.00,0.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,True
BUSN_REGN,0.0,0.75,0.75,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True
ENTN,0.0,0.80,0.80,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
GRADE,0.0,1.00,1.00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,True


In [ ]:
df.normal_data

0                                                         080068544,080134675,080135855,080139164,080147596
1                                                                                                       1,2
2                                                                                  0000,1234,1111,7168,1004
3                                                                                       담당자,김주희,정경민,김가영,이정희
4       2018-10-16 17:04:54,2019-07-15 14:10:07,2008-09-25 13:41:09,2008-09-26 15:21:20,2008-10-13 17:27:52
                                                       ...                                                 
2237                                                                      NULL,20180068,-,19910022,19960081
2239                                                                         NULL,자원순환지원부,청소행정과,환경관리과,자원순환과
2240                                                             NULL,1644-0007,02-3153-0514,-,031-590-0620
2241                        

In [61]:
pd.set_option('display.max.colwidth', None)
df.query('domain == "번호"')['normal_data']

0                             ['080068544', '080134675', '080135855', '080139164', '080147596']
2                                                      ['0000', '1234', '1111', '7168', '1004']
7                              ['NULL', '0000000000', '1208200052', '3068200471', '6138300022']
8                             ['131037196', '080146287', '200022389', '200100957', '202211277']
10                ['031-634-1762', '0629538902', '0319723084', '043-750-7395', '010-2542-4407']
                                                 ...                                           
2216                                                              ['NULL', '2', '3', '6', '10']
2231    ['210.99.81.253', '210.99.81.254', '210.99.81.251', '210.104.107.34', '210.104.107.50']
2233                          ['200305241', '200309912', '200311019', '200412103', '200418018']
2237                                          ['NULL', '20180068', '-', '19910022', '19960081']
2240                                 ['N

In [67]:
df.normal_data[0]

"['080068544', '080134675', '080135855', '080139164', '080147596']"